# Стиль

Помимо методов для работы с данными, `pandas` включает в себя возможности для форматирования таблиц!

Например: 

+ `df.style.highlight_null()` – подсветить ячейки с пропущенными значениями 
+ `df.style.highlight_max()` – подсветить ячейки с максимальными значениями по колонкам
+ `df.style.highlight_min()` – подсветить ячейки с минимальными значениями по колонкам
+ `df.style.applymap(func)` – применить стилевую функцию к каждой ячейке датафрэйма
+ `df.style.apply(func, axis, subset)` – применить стилевую функцию к каждой колонке/строке в зависимости от axis, subset позволяет выбрать часть колонок для оформления
+ `render()` – после декорирования возвращает HTML, описывающий табличку

Можно использовать несколько методов одновременно, применяя их друг за другом (method chaining). Давайте посмотрим на `style` подробнее в следующих шагах, а затем разберемся с тем, как можно отформатировать табличку с retention :)

# Индексы и подписи

Сначала создадим небольшой датафрейм:

In [4]:
import pandas as pd
import numpy as np

In [5]:
np.random.seed(77)
df = pd.DataFrame({'A': list(range(5)), 
                   'B': np.random.randint(0, 10, 5),
                   'C': np.random.randint(-10, 10, 5), 
                   'D': np.random.randint(-10, 100, 5)})
df

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


Первый метод – `.hide_index()`, позволяет спрятать индексы:

In [6]:
df.style.hide_index()

A,B,C,D
0,7,-10,49
1,4,-3,90
2,4,2,44
3,5,9,26
4,8,-10,37


Далее – `.set_caption()`. С его помощью можно добавить подпись к таблице:

In [7]:
df.style.hide_index().set_caption('Cool table')

A,B,C,D
0,7,-10,49
1,4,-3,90
2,4,2,44
3,5,9,26
4,8,-10,37


# Раскрашиваем ячейки
## highlight_min/max
`highlight_max` – подсвечивает (выделяет) цветом наибольшее значение. Можно применить либо к каждой строке (`axis=0/'index'`), либо к каждой колонке (`axis=1/'columns'`).

In [8]:
df.style.highlight_max(axis=1) 

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


In [9]:
df.style.highlight_max(axis='index')

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


Аналогичная функция для подсветки минимальных значений – `highlight_min()`.

In [11]:
df.style.highlight_min()

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


## background_gradient
`background_gradient` – раскрашивает ячейки в зависимости от их значений. В итоге получается что-то похожее на heatmap (тепловую карту). Например:

In [12]:
(df.style
 .highlight_min('A', color='red')
 .highlight_max('B', color='orange')
 .background_gradient(subset=['C','D'],cmap='viridis')
)

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


Здесь мы сначала выделяем красным минимальное значение в столбце `A`(`highlight_min`), затем – оранжевым максимальное в колонке `B` (`highlight_max`), и применяем `background_gradient` для `C` и `D`, указав палитру viridis. 

## style.bar
Визуализировать значения можно прямо в таблице с помощью `.bar()`. Данный метод принимает несколько аргументов:

+ `subset` – для каких колонок нужно построить небольшой барплот
+ `color` – цвет 

In [13]:
df.style.bar(subset=['C', 'D'], color='#67A5EB')

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


+ `align` –  как выровнять столбики (`mid` – центр ячейки в (max-min)/2; `zero` – ноль находится в центре ячейки; `left` – минимальное значение находится в левой части ячейки)

In [14]:
df.style.bar(subset=['C', 'D'], color='#67A5EB', align='mid')

,A,B,C,D
0,0,7,-10,49
1,1,4,-3,90
2,2,4,2,44
3,3,5,9,26
4,4,8,-10,37


Также можно указать сразу несколько цветов. Значения меньше 0 будут окрашены в красный, больше – в зелёный.

In [15]:
(df
 .style
 .hide_index()
 .bar(subset=['C'], align='mid',color=['#d65f5f', '#5fba7d'])
)

A,B,C,D
0,7,-10,49
1,4,-3,90
2,4,2,44
3,5,9,26
4,8,-10,37


# Форматирование отображения чисел
Иногда может понадобится различное число знаков после запятой. Для этого подходит метод `.format()`, которому нужно передать строку, указывающую сколько знаков необходимо оставить.

In [16]:
# генерируем данные
df = pd.DataFrame({'A': np.linspace(1, 10, 5)})
df = pd.concat([df, pd.DataFrame(np.random.randn(5, 4), columns=list('BCDE'))],axis=1)
df['F'] = np.random.choice(['A', 'B'], size=5)
df.iloc[3, 3] = np.nan 
df.iloc[0, 2] = np.nan 
df

,A,B,C,D,E,F
0,1.00,0.797939,NaN,-1.652119,0.717119,B
1,3.25,0.977228,-1.040849,-0.643520,-0.112520,A
2,5.50,-0.314166,1.627440,-0.361227,-0.173046,B
3,7.75,-1.951309,-0.978210,NaN,-1.178379,A
4,10.00,-0.515551,-0.063015,-0.559371,0.796697,A


Форматируем:

+ оставляем только 2 знака после точки
+ добавляем знак + для положительных значений
+ применяем ко всем колонкам, кроме F

In [17]:
df.style.format("{:+.2f}", subset=df.columns.drop('F'))

,A,B,C,D,E,F
0,+1.00,+0.80,+nan,-1.65,+0.72,B
1,+3.25,+0.98,-1.04,-0.64,-0.11,A
2,+5.50,-0.31,+1.63,-0.36,-0.17,B
3,+7.75,-1.95,-0.98,+nan,-1.18,A
4,+10.00,-0.52,-0.06,-0.56,+0.80,A


Также можем скрыть индексы и добавить название:

In [18]:
(df.style
 .format({'B': "{:0<4.0f}", 'D': '{:+.2f}'})
 .hide_index()
 .set_caption('Новая таблица'))

A,B,C,D,E,F
1.000000,1000,nan,-1.65,0.717119,B
3.250000,1000,-1.040849,-0.64,-0.112520,A
5.500000,-000,1.627440,-0.36,-0.173046,B
7.750000,-200,-0.978210,+nan,-1.178379,A
10.000000,-100,-0.063015,-0.56,0.796697,A


И при желании импортировать в Excel (но не всё форматирование переносится):

In [19]:
(df.style
 .bar(align='mid', color=['#d65f5f', '#5fba7d'])
 .to_excel('styled.xlsx', engine='openpyxl')
)

# retention 
**Retention** – показатель удержания пользователей. Иными словами – отражает то, сколько пользователей возвращаются в продукт спустя заданное время. 

Обычно день начала использования сервиса называется Day 0 – момент, когда юзер впервые воспользовался продуктом. N-Day Retention показывает, сколько процентов пользователей, начавших пользоваться продуктом в день 0, вернулись и продолжили использовать продукт N дней спустя.  

Под днем не всегда понимается день – интервалы измерения ретеншена зависят от характеристик самого продукта. Так, некоторые сервисы подразумевают ежедневное использование (напр. социальные сети), а другие – более редкое (бронирование, такси, доставка, рестораны). Согласитесь, вряд ли пользователи бронируют авиабилеты или отели каждый день :) В случае сервиса доставки или ресторана, мы могли бы посмотреть на недельные интервалы использования (week by week). Тогда retention бы показывал, сколько пользователей вернулись в 1-7 день, 8-14 и т.д.

## визуализация
Один из вариантов визуализации представлен ниже. Что же здесь происходит?

+ `Cohort`, строки – когорта пользователей. Например, 2011-01 означает, что пользователи из этой группы первый раз сделали заказ в онлайн магазине в январе 2011 года, 2011-02 – в феврале, и т.д.
+ `CohortPeriod`, столбцы – месяц. 0 – когда пользователи только-только сделали первую покупку. Далее – сколько из них оформили заказ в 1 месяце, 2, ..., 12-м. Часть значений остается пропущенной, поскольку период наблюдений для части пользователей меньше, чем для остальных. Для юзеров, присоединившихся в декабре 2010, имеются данные за весь год, в то время как для ребят из когорты 2011-11 – всего лишь за 0 и 1 месяц.

Загрузить исходную табличку можно [отсюда](https://stepik.org/media/attachments/lesson/367416/user_retention.csv)

In [22]:
user_retention = pd.read_csv("https://stepik.org/media/attachments/lesson/367416/user_retention.csv", index_col=0)

In [23]:
user_retention

,0,1,2,3,4,5,6,7,8,9,10,11,12
Cohort,,,,,,,,,,,,,
2010-12,1.0,0.381857,0.334388,0.387131,0.359705,0.396624,0.379747,0.354430,0.354430,0.394515,0.373418,0.500000,0.274262
2011-01,1.0,0.239905,0.282660,0.242280,0.327791,0.299287,0.261283,0.256532,0.311164,0.346793,0.368171,0.149644,NaN
2011-02,1.0,0.247368,0.192105,0.278947,0.268421,0.247368,0.255263,0.281579,0.257895,0.313158,0.092105,NaN,NaN
2011-03,1.0,0.190909,0.254545,0.218182,0.231818,0.177273,0.263636,0.238636,0.288636,0.088636,NaN,NaN,NaN
2011-04,1.0,0.227425,0.220736,0.210702,0.207358,0.237458,0.230769,0.260870,0.083612,NaN,NaN,NaN,NaN
2011-05,1.0,0.236559,0.172043,0.172043,0.215054,0.243728,0.265233,0.103943,NaN,NaN,NaN,NaN,NaN
2011-06,1.0,0.208511,0.187234,0.272340,0.246809,0.336170,0.102128,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,1.0,0.209424,0.204188,0.230366,0.272251,0.115183,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,1.0,0.251497,0.251497,0.251497,0.137725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Довольно сложно воспринимать подобную информацию без цвета, поэтому применяем рассмотренные ранее методы:

In [24]:
ur_style = (user_retention
            .style
            .set_caption('User retention by cohort')  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
ur_style

,0,1,2,3,4,5,6,7,8,9,10,11,12
Cohort,,,,,,,,,,,,,
2010-12,100.00%,38.19%,33.44%,38.71%,35.97%,39.66%,37.97%,35.44%,35.44%,39.45%,37.34%,50.00%,27.43%
2011-01,100.00%,23.99%,28.27%,24.23%,32.78%,29.93%,26.13%,25.65%,31.12%,34.68%,36.82%,14.96%,
2011-02,100.00%,24.74%,19.21%,27.89%,26.84%,24.74%,25.53%,28.16%,25.79%,31.32%,9.21%,,
2011-03,100.00%,19.09%,25.45%,21.82%,23.18%,17.73%,26.36%,23.86%,28.86%,8.86%,,,
2011-04,100.00%,22.74%,22.07%,21.07%,20.74%,23.75%,23.08%,26.09%,8.36%,,,,
2011-05,100.00%,23.66%,17.20%,17.20%,21.51%,24.37%,26.52%,10.39%,,,,,
2011-06,100.00%,20.85%,18.72%,27.23%,24.68%,33.62%,10.21%,,,,,,
2011-07,100.00%,20.94%,20.42%,23.04%,27.23%,11.52%,,,,,,,
2011-08,100.00%,25.15%,25.15%,25.15%,13.77%,,,,,,,,


Отлично! Теперь довольно легко заметить, что ретеншен в каждый из месяцев был наибольшим среди пользователей из самой первой когорты, 2010-12. Подумайте, что может влиять на подобный показатель (e.g. какие изменения).